
# Introducción a Pandas

Pandas es una librería de Python utilizada para tratar datos en forma de tabla. Nos permite importar, exportar y hacer las operaciones habituales que nos permiten otras herramientas como Excel o el lenguaje SQL

## Importación y exportación

Podemos importar datos a DataFrames de Pandas de diferentes orígenes y formatos, entre ellos:

* De CSV: con [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)
* De Excel: con [`read_excel`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html)
* De base de datos: con [`read_sql`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html)


Escribir en destinos de datos


Como ejemplo, vamos a importar unos datos que tenemos en CSV con precios de alquileres en los distritos de Madrid:

In [1]:
import pandas as pd

alquiler = pd.read_csv('dat/alquiler-madrid-distritos.csv', index_col=False)
alquiler.head()

,distrito,ano,quarter,precio
0,Arganzuela,2007,2,13.066587
1,Barajas,2007,2,11.199855
2,Carabanchel,2007,2,11.127661
3,Centro,2007,2,17.746404
4,Chamartín,2007,2,14.386480


También podemos exportar esos datos a CSV haciendo:

In [2]:
alquiler.to_csv('alquiler.csv')

## Estructura básica e inspección

Las tablas en `pandas` son objetos de la clase `DataFrames`. Un `DataFrame` consta de dos partes: un índice y los datos propiamente dichos. Las columnas de los datos son de la clase `Series`.

Para consultar las columnas de un `DataFrame`, accedemos a la propiedad `columns`.

In [3]:
alquiler.columns

Index(['distrito', 'ano', 'quarter', 'precio'], dtype='object')

Si además queremos saber el tipo del dato, accedemos a la propiedad `dtypes`.

*Nota*: las cadenas de texto se marcan como `object` dentro de un DataFrame

In [4]:
alquiler.dtypes

distrito     object
ano           int64
quarter       int64
precio      float64
dtype: object

Cada `DataFrame` tiene un índice. Si no lo hemos especificado, será un incremental sin relación con nuestros datos. El uso de índices está recomendado cuando tratamos con datos grandes, ya que permite acceder a las filas por _hash_ en lugar de tener que iterar por todas ellas para encontrar el valor que se busca. Los índices también son importantes a la hora de realizar agregaciones y cruces entre tablas.

Para consultar cuál es el índice de un DataFrame, accedemos a la propiedad `index`.

In [5]:
alquiler.index

RangeIndex(start=0, stop=840, step=1)

Podemos alterarlo con `set_index`. El nuevo índice puede ser una o varias columnas.

In [6]:
alquiler_nuevo_indice = alquiler.set_index(['distrito', 'ano', 'quarter'])

Una forma rápida de echar un vistazo a los datos es consultas las primeras o últimas filas del DataFrame, con las funciones `head` y `tail`.

In [7]:
alquiler.head()

,distrito,ano,quarter,precio
0,Arganzuela,2007,2,13.066587
1,Barajas,2007,2,11.199855
2,Carabanchel,2007,2,11.127661
3,Centro,2007,2,17.746404
4,Chamartín,2007,2,14.386480


In [8]:
alquiler.tail()

,distrito,ano,quarter,precio
835,Tetuan,2018,2,15.114558
836,Usera,2018,2,11.533458
837,Vicálvaro,2018,2,9.962139
838,Villa De Vallecas,2018,2,10.915967
839,Villaverde,2018,2,10.427527


Podemos seleccionar un listado de columnas a devolver de la siguiente forma:

In [9]:
alquiler[['distrito', 'precio']].head()

,distrito,precio
0,Arganzuela,13.066587
1,Barajas,11.199855
2,Carabanchel,11.127661
3,Centro,17.746404
4,Chamartín,14.386480


Para conocer el número de filas de una tabla hay varias opciones:

In [10]:
len(alquiler)

840

In [11]:
alquiler.shape

(840, 4)

##### Nota

El índice no forma parte propiamente de los datos:

In [ ]:
alquiler_nuevo_indice.shape

## Filtro y selección

Hay tres operadores fundamentales para seleccionar filas y columnas: `loc`, `iloc` y `[]`. La diferencia fundamental entre `loc` e `iloc` es que el primero requiere _etiquetas_ y el segundo, índices numéricos (la `i` inicial viene de `integer`).


### Selección por índices numéricos

Para acceder por posición usando índices numéricos, se usa `iloc[]`, como en los siguientes ejemplos:

In [12]:
# por defecto, seleccionamos filas
alquiler_nuevo_indice.iloc[200]

precio    9.901414
Name: (Moratalaz, 2010, 4), dtype: float64

In [13]:
# pero también se pueden seleccionar filas y columnas
# además, usando rangos
alquiler.iloc[3:5, 1:]

,ano,quarter,precio
3,2007,2,17.746404
4,2007,2,14.386480


In [ ]:
# índices no consecutivos
# recuerda: en python, se empieza a contar en 0
alquiler.iloc[[1, 2, 4], [0, 3]]

In [ ]:
# los índices negativos indican que se empieza a contar desde el final
alquiler.iloc[-3:-1]

### Selección por etiquetas

Para acceder por _etiquetas_ (es decir, columnas parte del índice), se usa `loc[]`

In [14]:
alquiler_nuevo_indice.loc[('Centro', 2014, 2)]

precio    13.39
Name: (Centro, 2014, 2), dtype: float64

In [15]:
# O un distrito completo
alquiler_nuevo_indice.loc[('Centro')].head()

precio
ano  quarter           
2007 2        17.746404
     4        18.044594
2008 2        17.618608
     4        16.858053
2009 2        15.788713

### Selección por condiciones

Para extraer las filas que cumplen una condición, le pasamos al DataFrame una Series de booleanos, o directamente algo que la devuelva.

In [16]:
alquiler[alquiler.distrito == 'Retiro'].head()

,distrito,ano,quarter,precio
13,Retiro,2007,2,13.747522
34,Retiro,2007,4,14.021948
55,Retiro,2008,2,14.240559
76,Retiro,2008,4,13.974290
97,Retiro,2009,2,13.019647


**Nota:** mira cómo en el código anterior hemos seleccionado la columna `distrito` usando la sintaxis sumamente compacta `alquiler.distrito`.

Podemos combinar varias condiciones con `&` (y lógico) y `|` (o lógico)

In [17]:
# No olvides los paréntesis, es importante por prioridad de operandos!

alquiler[(alquiler.distrito == 'Retiro') & (alquiler.ano == 2012)]

,distrito,ano,quarter,precio
307,Retiro,2012,1,12.601891
328,Retiro,2012,2,12.363832
349,Retiro,2012,3,12.167435
370,Retiro,2012,4,11.951317


## Ordenación

Podemos ordenar un DataFrame por una o varias columnas, de forma ascendente o descendente, con [`sort_values`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html)

In [18]:
alquiler.sort_values('distrito', ascending=True).head()

,distrito,ano,quarter,precio
0,Arganzuela,2007,2,13.066587
504,Arganzuela,2014,3,10.774739
483,Arganzuela,2014,2,10.680000
462,Arganzuela,2014,1,10.937478
441,Arganzuela,2013,4,10.885807


In [ ]:
alquiler.sort_values(['ano', 'quarter', 'distrito'], ascending=[False, False, True]).head()

## Transformación

Nuevas columnas calculadas, cambio de tipo de dato, eliminar una columna

Podemos operar sobre las columnas para crear otras nuevas o cambiar el tipo de dato

In [19]:
# Hago una copia para no modificar el dataframe original
alquiler_2 = alquiler.copy()

alquiler_2['precio_90m'] = alquiler_2.precio * 90
alquiler_2.head()

,distrito,ano,quarter,precio,precio_90m
0,Arganzuela,2007,2,13.066587,1175.992857
1,Barajas,2007,2,11.199855,1007.986923
2,Carabanchel,2007,2,11.127661,1001.489519
3,Centro,2007,2,17.746404,1597.176343
4,Chamartín,2007,2,14.386480,1294.783156


Las operaciones que no se pueden lanzar directamente sobre la `Series` completa, la ejecutamos por elemento utilizando `apply`

In [20]:
# Fíjate bien en la función lambda, es una función en una sola línea
alquiler_2['ano_quarter'] = alquiler_2.apply(lambda fila: str(fila.ano) + 'Q' + str(fila.quarter), axis=1)
alquiler_2.head()

,distrito,ano,quarter,precio,precio_90m,ano_quarter
0,Arganzuela,2007,2,13.066587,1175.992857,2007Q2
1,Barajas,2007,2,11.199855,1007.986923,2007Q2
2,Carabanchel,2007,2,11.127661,1001.489519,2007Q2
3,Centro,2007,2,17.746404,1597.176343,2007Q2
4,Chamartín,2007,2,14.386480,1294.783156,2007Q2


## Resumen estadístico

Pandas provee una serie de funciones de resumen estadístico que podemos aplicar sobre una columna concreta, o sobre todas las del DataFrame.

Para un resumen para todas las columnas de número de filas, media, desviación estándar, cuartiles, ... usamos [`describe`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html)

In [21]:
alquiler.describe()

,ano,quarter,precio
count,840.00000,840.00000,840.000000
mean,2012.90000,2.52500,11.545779
std,3.08242,1.11842,2.271275
min,2007.00000,1.00000,7.591541
25%,2010.75000,2.00000,9.853159
50%,2013.00000,2.00000,11.132781
75%,2015.25000,4.00000,12.996690
max,2018.00000,4.00000,19.308607


Podemos utilizar también `sum`, `mean`, `std`, `count`, `min`, `max`, ... sobre el DataFrame o una columna en concreto

In [22]:
alquiler.mean()

ano        2012.900000
quarter       2.525000
precio       11.545779
dtype: float64

In [23]:
alquiler.precio.max()

19.308606815183698

## Agrupación

De una forma equivalente a como hacemos en SQL, podemos agregar las tablas y sacar resúmenes de los grupos. La operación en pandas se hace en dos fases:

* El `groupby`: donde especificamos la o las columnas por las que agregar
* La aplicación de la función de agregación sobre una o varias columnas

Un resumen usando una función de agregación sobre todas las columnas del DataFrame

In [24]:
alquiler.groupby('ano').max()

# Atención, fíjate bien en lo que hace esto. Saca el valor máximo de distrito (alfabéticamente),
#  de quarter y precio (numéricamente), pero no representa filas completas
# Es decir, Villaverde en el quarter 4 no tuvo ese precio

,distrito,quarter,precio
ano,,,
2007,Villaverde,4,18.044594
2008,Villaverde,4,17.618608
2009,Villaverde,4,15.788713
2010,Villaverde,4,15.564614
2011,Villaverde,4,15.078333
2012,Villaverde,4,14.822377
2013,Villaverde,4,13.704467
2014,Villaverde,4,13.965912
2015,Villaverde,4,15.251220


Para hacerlo únicamente sobre una columna:

In [25]:
alquiler.groupby('ano').precio.min()

ano
2007     9.592651
2008    10.628444
2009     9.764769
2010     9.279254
2011     9.089262
2012     8.693941
2013     7.933318
2014     7.591541
2015     7.754823
2016     8.183480
2017     9.210458
2018     9.962139
Name: precio, dtype: float64

Para aplicar diferentes resúmenes sobre diferentes columnas

In [26]:
tmp = alquiler.groupby('ano').agg({'precio': 'mean', 'distrito': 'first'})
tmp.head()

,precio,distrito
ano,,
2007,12.679226,Arganzuela
2008,12.963422,Arganzuela
2009,11.900691,Arganzuela
2010,11.608368,Arganzuela
2011,11.461966,Arganzuela


## Cruce

Podemos cruzar dos tablas por una o varias columnas en pandas, de forma equivalente a como hacemos en SQL, con [`merge`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html). También podemos usar los distintos tipos de cruce:

* `inner`: para obtener solamente los registros que crucen en ambas tablas
* `left` o `right`: para mantener los registros de una de las dos tablas, crucen o no con la otra
* `outer`: para manter los registros de ambas tablas, crucen o no

In [27]:
# Aquí, además, un ejemplo de cómo crear un dataframe a partir de un diccionario
df_ejemplo = pd.DataFrame({'distrito':  ['Moratalaz', 'Centro', 'Barajas'],
                           'poblacion': [95000, 150000, 46000]})
df_ejemplo

,distrito,poblacion
0,Moratalaz,95000
1,Centro,150000
2,Barajas,46000


In [28]:
tmp = df_ejemplo.merge(alquiler, on='distrito')
tmp.head()

,distrito,poblacion,ano,quarter,precio
0,Moratalaz,95000,2007,2,11.498611
1,Moratalaz,95000,2007,4,11.117894
2,Moratalaz,95000,2008,2,11.925685
3,Moratalaz,95000,2008,4,10.966299
4,Moratalaz,95000,2009,2,10.229014


In [29]:
len(tmp)

120

In [30]:
tmp = df_ejemplo.merge(alquiler, on='distrito', how='right')
tmp.tail()

,distrito,poblacion,ano,quarter,precio
835,Villaverde,NaN,2017,2,9.537615
836,Villaverde,NaN,2017,3,9.980975
837,Villaverde,NaN,2017,4,10.296415
838,Villaverde,NaN,2018,1,10.230638
839,Villaverde,NaN,2018,2,10.427527


In [ ]:
len(tmp)